In [62]:
import numpy as np
import pandas as pd
import os

In [63]:
filename=os.path.join('crowdstorm_disaggregated.csv') 
df = pd.read_csv(filename)

In [64]:
df.columns

Index(['playerShort', 'player', 'club', 'leagueCountry', 'birthday', 'height',
       'weight', 'position', 'games', 'victories', 'ties', 'defeats', 'goals',
       'yellowCards', 'yellowReds', 'redCards', 'photoID', 'rater1', 'rater2',
       'refNum', 'refCountry', 'Alpha_3', 'meanIAT', 'nIAT', 'seIAT',
       'meanExp', 'nExp', 'seExp', 'skintone', 'allreds', 'allredsStrict',
       'refCount'],
      dtype='object')

In [65]:
# TODO: Improve data preparation!
# take average of the two raters
df["rating"] = (df["rater1"] + df["rater2"]) / 2
# preliminary selection of features
df = df[["rating", "height", "weight", "games", "victories", "ties", "defeats", "goals", "yellowCards", "yellowReds", "redCards"]]
# drop instances with undefined values => check if there aer better options: https://en.wikipedia.org/wiki/Missing_data
df = df.dropna()

In [66]:
x = np.array(df[["rating", "height", "weight", "games", "victories", "ties", "defeats", "goals", "yellowCards", "yellowReds", "redCards"]])
y = np.array(df["redCards"] / df["games"]) # target should be the fraction of games with red cards

In [67]:
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [68]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=0)

In [69]:
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)
prediction = reg.predict(X_test)
mean_squared_error(y_test, prediction)

0.0007450238012380009

In [70]:
regr = RandomForestRegressor()
regr = regr.fit(X_train, y_train)
prediction = regr.predict(X_test)
mean_squared_error(y_test, prediction)

4.890885096727856e-07

In [71]:
errors1 = []
errors2 = []
rating = x[:,0]
for it in range(19):
    randomRating = np.random.permutation(rating)
    data = x.copy()
    data[:,0] = randomRating

    X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.4, random_state=0)

    reg = linear_model.LinearRegression()
    reg.fit(X_train, y_train)
    prediction = reg.predict(X_test)

    error = mean_squared_error(y_test, prediction)
    errors1.append(error)

    # Takes very long (ca. 5min)
    #regr = RandomForestRegressor()
    #regr = regr.fit(X_train, y_train)
    #prediction = regr.predict(X_test)

    error = mean_squared_error(y_test, prediction)
    errors2.append(error)

print(np.average(errors1))
print(np.average(errors2))

0.0007450214748805821
0.0007450214748805821
